## 당뇨병 여부 판단
- 이상치 처리 (Glucose, BloodPressure, SkinThickness, Insulin, BMI가 0인 값)

## [참고]작업형2 문구
- 출력을 원하실 경우 print() 함수 활용
- 예시) print(df.head())
- getcwd(), chdir() 등 작업 폴더 설정 불필요
- 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

### 데이터 파일 읽기 예제
- import pandas as pd
- X_test = pd.read_csv("data/X_test.csv")
- X_train = pd.read_csv("data/X_train.csv")
- y_train = pd.read_csv("data/y_train.csv")

### 사용자 코딩

### 답안 제출 참고
- 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
- pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('003000000.csv', index=False)

In [ ]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Outcome')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

## Start

## 라이브러리 및 데이터 불러오기

In [ ]:
# 라이브러리
import pandas as pd

In [ ]:
# 데이터 불러오기 (생략)
X_train.shape, X_test.shape, y_train.shape

## EDA

In [ ]:
X_train.head(3)

In [ ]:
y_train.head(3)

In [ ]:
X_test.head(3)

In [ ]:
X_train.describe()

In [ ]:
X_train.isnull().sum(), X_test.isnull().sum(), y_train.isnull().sum(), 

In [152]:
y_train['Outcome'].value_counts()

0    400
1    209
Name: Outcome, dtype: int64

## 데이터 전처리

In [153]:
#이상치 처리
#Train
print("Glucose: ", len(X_train[X_train['Glucose'] == 0]))
print("BloodPressure: ", len(X_train[X_train['BloodPressure'] == 0]))
print("SkinThickness: ", len(X_train[X_train['SkinThickness'] == 0]))
print("Insulin: ", len(X_train[X_train['Insulin'] == 0]))
print("BMI: ", len(X_train[X_train['BMI'] == 0]))

Glucose:  0
BloodPressure:  0
SkinThickness:  0
Insulin:  0
BMI:  0


In [154]:
#Test
print("Glucose: ", len(X_test[X_test['Glucose'] == 0]))
print("BloodPressure: ", len(X_test[X_test['BloodPressure'] == 0]))
print("SkinThickness: ", len(X_test[X_test['SkinThickness'] == 0]))
print("Insulin: ", len(X_test[X_test['Insulin'] == 0]))
print("BMI: ", len(X_test[X_test['BMI'] == 0]))

Glucose:  0
BloodPressure:  0
SkinThickness:  0
Insulin:  0
BMI:  0


트레인에만 0이 있는 포도당(Glucose)는 삭제
나머지는 평균값으로 대체 하여 이상치를 처리함

In [155]:
# 포도당 이상치 삭제
print(X_train.shape, y_train.shape)
del_index = X_train[X_train['Glucose'] == 0].index
X_train = X_train.drop(index = del_index, axis=0)
y_train = y_train.drop(index = del_index, axis=0)
print(X_train.shape, y_train.shape)

(609, 9) (609, 2)
(609, 9) (609, 2)


In [156]:
# 포도당을 제외한 이상치, 평균값으로 대체
columns = ['BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
col_means = X_train[columns].mean()
X_train[columns] = X_train[columns].replace(0, col_means)
print("BloodPressure: ", len(X_train[X_train['BloodPressure'] == 0]))

BloodPressure:  0


In [157]:
# 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
cols = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']
X_train[cols] = scaler.fit_transform(X_train[cols])
X_test[cols] = scaler.fit_transform(X_test[cols])

In [158]:
# id 제외
X = X_train.drop('id',axis=1)
test = X_test.drop('id', axis=1)

In [159]:
from sklearn.svm import SVC
model = SVC(random_state=2022)
model.fit(X, y_train['Outcome'])
predictions = model.predict(test)

In [160]:
round(model.score(X, y_train['Outcome']) * 100, 2) 

82.92

In [162]:
output = pd.DataFrame({'idx': X_test.index, 'Outcome': predictions})
output.head()

,idx,Outcome
0,258,1
1,220,1
2,438,0
3,130,1
4,730,0


In [163]:
# 수험번호.csv로 출력
output.to_csv('1234567.csv', index=False)

## 결과 체점 (수험자는 알 수 없는 부분임)

In [164]:
round(model.score(test, y_test['Outcome']) * 100, 2) # 60점대로 낮은 정확도를 보여줌

75.32